In [1]:
%load_ext Cython

import pandas as pd
import numpy as np
import datetime
import random
import pickle
from surprise import SVD
from surprise import Dataset
from surprise import Reader
import statistics

from explanation import InfluenceExplainer

In [2]:
from recommender_algo.editable_svd import EditableSVD

In [3]:
# Load pre-trained model
with open("algo-20m.pickle", "rb") as fp:
    algo = pickle.load(fp)
    
# Init list of movies
movies_df = pd.read_csv("../data/ml-20m/movies.csv", dtype={
   'movieId': int,
   'title': str,
   'genres': str,
})
movies_map = {}
for index, row in movies_df.iterrows():
    movies_map[row['movieId']] = row['title']

user_id = 548328920
means_list = []
std_list = []
for i in range(100):
    # create 10 ratings for random films
    movies_to_rate = []
    for k in range(10):
        key = random.choice(list(movies_map.keys()))
        movies_to_rate.append(key)
    user_rated_items = { key: random.randint(1, 5) for key in movies_to_rate }
    print(user_rated_items)
    
    # Generate recommendations
    algo.fit_new_user(user_id, user_rated_items)
    top_n = []
    for i in movies_map:
        # Filter out rated movies
        if i not in user_rated_items:
            prediction = algo.predict(user_id, i)
            top_n.append((prediction.iid, prediction.est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    top_n.sort(key=lambda x: x[1], reverse=True)
    top_n = top_n[:6]
    print(top_n)
    
    influence_explainer = InfluenceExplainer(user_id, user_rated_items, algo)
    
    for recommendation in top_n:        
        film_influences = {}
        for k in range(20):
            explanation = influence_explainer.generate_explanation_for_all(recommendation)
            for (key, influence) in explanation:
                if key not in film_influences:
                    film_influences[key] = []
                film_influences[key].append(influence)
        # get mean, std of calculated influences
        for influences_list in film_influences.values():
            means_list.append(statistics.mean(influences_list))
            std_list.append(statistics.stdev(influences_list))
    algo.delete_user(user_id)
print("Mean: " + str(statistics.mean(means_list)) + ", std: " + str(statistics.stdev(std_list)))

{2303: 5, 59465: 2, 39715: 1, 27455: 2, 93193: 2, 117861: 5, 78105: 3, 1791: 4, 101433: 4, 249: 5}
[(2905, 4.651711360984757), (5971, 4.638952178171398), (108583, 4.623508709804668), (1289, 4.620384470757121), (82143, 4.576429580684041), (44555, 4.575887118779923)]
{98400: 2, 106421: 4, 2910: 1, 94153: 4, 93408: 1, 122519: 2, 120110: 1, 97757: 2, 3874: 2, 1242: 5}
[(318, 3.9525394254622395), (2324, 3.903208268481807), (58559, 3.8535935514096833), (527, 3.823416818758369), (79132, 3.8122755433216584), (91529, 3.8009480755953002)]
{104817: 2, 4934: 1, 71327: 5, 87446: 4, 107368: 1, 129344: 2, 100173: 5, 128383: 2, 95105: 3, 54229: 3}
[(26150, 4.3555548223356055), (1183, 4.3358182919269606), (2731, 4.284301470635518), (7938, 4.263258175444254), (100553, 4.252453754349466), (296, 4.247432341937645)]
{6293: 2, 8294: 1, 6667: 3, 97662: 1, 2709: 5, 26480: 2, 65357: 2, 103472: 2, 7878: 3, 109168: 5}
[(4308, 4.278618697214631), (6942, 4.082348739744419), (34405, 4.024699141497105), (100553, 4.0

In [4]:
with open("inf1_means_list.pickle", "wb+") as fp:
    pickle.dump(means_list, fp)
with open("inf1_std_list.pickle", "wb+") as fp:
    pickle.dump(std_list, fp)

In [5]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 22})

fig, ax = plt.subplots(figsize=(20,10))
ax.set_title('Mean of repeated influence calculations by a rated film on recommendation')
ax.hist(means_list, bins=20)
plt.xlabel('Mean influence (stars)')
fig.savefig('exp1_mean-hist.png')

fig, ax = plt.subplots(figsize=(20,10))
ax.set_title('Standard deviation of repeated influence calculations by a rated film on recommendation')
ax.hist(std_list, bins=20)
plt.xlabel('Std of influence (stars)')
fig.savefig('exp1_std-hist.png')